In [ ]:
class TestManagement():
    def __init__(self):
        self.testOutputZeros = {}
        self.testOutputOnes = {}

    def store_random_data(self, key, randomData):
        self.testOutputZeros[key] = randomData.count('0')
        self.testOutputOnes[key] = randomData.count('1')

    def basic_prediction(self, key):
        predictOneSuccess = self.testOutputOnes[key] / (self.testOutputZeros[key] + self.testOutputOnes[key])
        predictZeroSuccess = 1 - predictOneSuccess
        return max(predictOneSuccess, predictZeroSuccess)

    def check_bias(self, key):
        predictOneSuccess = self.testOutputOnes[key] / (self.testOutputZeros[key] + self.testOutputOnes[key])
        predictZeroSuccess = 1 - predictOneSuccess
        print("'1' character percentage:", predictOneSuccess)
        print("'0' character percentage:", predictZeroSuccess)
            


def CrudePRNG_output(self, seed=0, length=100):
    nextBit = seed%2
    if (nextBit == 1):
        output = "10" * int(length/2)
    else:
        output = "01" * int(length/2)

    if (length%2 == 1):
        output += str(nextBit)
        if (nextBit == 0): nextBit = 1
        else: nextBit = 0
    
    return output
